## 2lvl or 3lvl Buck or Buckboost Power Dissapation

#### to do:
-change mosfet.Fet_cap_vs_vds.cgd_0V back
-change mosfet.Fet_cap_vs_vds.c_gs back

In [1]:
import libs.append_path

# from circuit_4state import circuit_params as circuit_params_4state
# from circuit_2state import circuit_params as circuit_params_2state
# from controller import get_ic_params
# from cyntec import Inductor_pdis, create_ind_family_df, ind_pdis_obj, l_set, cyntec_filename
# import mosfet_hs
# import mosfet_ls # import get_fet_params, Fet_cap_vs_vds, Fet_switching_on, Fet_switching_off, Losses
# from unit_waveforms_turnon import Four_state
# from capacitors import Caplosses
#from board import p_cu

import pd_filter_fcns

from bucklosses_dcm import Buckconverter_losses

### Input Parameters

In [ ]:
input_params = {'vin': 13.2,
                'vout': 5,
                'pin': 5*16, 
                'eff': 1,
                'fs':375e3,
                'tambient':65,
                'controller':'raa489300',
                'lout':{'family':'cmll063t',
                        'value(uH)':0.4,
                        'config':'single'},
                'lvl_config':'3 level',
                'hsfet_partnum':'SISH536DN',
                'lsfet_partnum':'SISS52DN',
                'vgate':5,
                'm_hs':1,
                'm_ls':1,
                'rd':0.5,
                'caps':{'vin'   :{'partnum':'GRM188C81C106MA73','n':4},
                        'vout'  :{'partnum':'GRM155C80J106ME11D','n':10},
                        'flying':{'partnum':'GRM188C81C106MA73','n':4}}
               }
input_params['iout'] = input_params['pin']*input_params['eff']/input_params['vout']

In [ ]:
input_params = {'vin': 48,
                'vout': 20,
                'pin': 241,
                'eff': 1,
                'fs':315e3,
                'tambient':45,
                'controller':'raa489300',
                'lout':{'family':'hbed053t',
                        'value(uH)':1.0,
                        'config':'single'},
                'lvl_config':'3 level',
                'hsfet_partnum':'ISZ0702NLS',
                'lsfet_partnum':'BSZ024N04LS6',
                'q4_partnum':'SHORT',
                'vgate':10,
                'm_hs':1,
                'm_ls':1,
                'rd':0.5,
                'caps':{'vin'   :{'partnum':'GRM32EC72A106KE05L','n':4},
                        'vout'  :{'partnum':'GRM32EC72A106KE05L','n':8},
                        'flying':{'partnum':'C2012X5R1H106K125AC','n':8}}
               }
input_params['iout'] = input_params['pin']*input_params['eff']/input_params['vout']

In [2]:
input_params = {'vin': 28,
                'vout': 10,
                'pin': 140,
                'eff': .95,
                'fs':600e3,
                'tambient':45,
                'controller':'raa489110',
                'r_shunt_input':0.005,
                'lout':{'family':'hbed063t',
                        'value(uH)':2.2,
                        'config':'parallel'},
                'lvl_config':'2 level',
                'hsfet_partnum':'SIRA74DP',#'AONR66406',#'SIS488DN',#'SISS4410DN',#'SISA14DN',##'SISS4410DN',#'BSC059N04LS6', #'SIS488DN''SIR426DP'
                'lsfet_partnum':'SIRA74DP',#'SISA72ADN',#'AONS66408',#'SISS4410DN',#'SIRA74DP', #'AONS66408',#'BSC059N04LS6',#'BSZ024N04LS6',#'SIRA74DP',
                'q4_partnum':'AON7318',#'SISA14DN',#'SISA14DN', #'SHORT', #'SISS52DN',
                'vgate':5,
                'm_hs':1,
                'm_ls':1,
                'rd':0.5,
                'caps':{'vin'   :{'partnum':'GRM32EC72A106KE05L','n':4},
                        'vout'  :{'partnum':'GRM32EC72A106KE05L','n':8},
                        'flying':{'partnum':'C2012X5R1H106K125AC','n':8}}
               }
input_params['iout'] = input_params['pin']*input_params['eff']/input_params['vout']

## Need to add 
1. input shunt conduction losses
2. self heating mosfets

In [3]:
b_loss = Buckconverter_losses(input_params)
b_loss.p_totals|b_loss.p_summary

{'hs fet': 2.427,
 'ls fet': 0.965,
 'q4 fet': 0.484,
 'lout': 1.001,
 'caps': 0.125,
 'ic_with_gate': 0.218,
 'total': 6.386,
 'efficiency': 0.954,
 'hs turn-on': 0.468,
 'hs turn-off': 1.567,
 'hs rdson': 0.353,
 'hs ringing': 0.038,
 'hs gate': 0.104,
 'ls rdson': 0.636,
 'ls bd': 0.194,
 'ls qrr': 0.136,
 'ls gate': 0.104,
 'q4_rdson': 0.484,
 'lout rdc+rac': 0.579,
 'lout core': 0.422,
 'flying cap': 0.0,
 'input cap': 0.124,
 'board cu': 0.051,
 'ic': 0.01,
 'inp_shunt': 0.113}

In [ ]:
descr=b_loss.p_totals.keys()
pdis=b_loss.p_totals.values()

In [ ]:
df=pd.DataFrame.from_dict(b_loss.p_totals,orient='index',columns=['pdis'])
df

In [ ]:
fig=px.bar(df,orientation='h')
fig.show()

In [ ]:
df=pd.DataFrame.from_dict(b_loss.p_summary,orient='index',columns=['pdis'])
fig=px.bar(df,orientation='h')
fig.show()

## TO DO:  

In [ ]:
idx = df.index
val = df['pdis'].values
val_rev = np.flip(val)

In [ ]:
fig = make_subplots(
    rows=1,cols=2,
    column_widths=[.27,.73],
    horizontal_spacing = .04,
    specs=[[{"type":"table"},{"type":"bar"}]]
)
fig.add_trace(
    go.Table(
        header = dict(
            values=[' ',' '],
            height=1,
            font=dict(size=30)
        ),
        # header = dict(
        #     values = ['index','value'],
        #     font=dict(size=20),
        #     align="left"
        # ),
        cells = dict(
            values = [idx,val],
            font=dict(size=20),
            height=31,
            align="left"
        )
        
    ),
    
    row=1,col=1
)
fig.add_trace(
    go.Bar(
        x=val_rev,
        orientation='h'
    ),
    row=1,col=2
)
fig.update_layout(
    height=650,
    showlegend=False,
    title_text="Power Loss by Component",
)

fig.show()

In [ ]:
b_loss.p_summary|b_loss.p_totals

In [ ]:
var = {'param':'iout','value':17}

In [ ]:
def b_obj_var(inp_params:dict,variable:dict):
    p = inp_params
    p[variable['param']] = variable['value']
    return Buckconverter_losses(p) 

def df_var(inp_params,variable:dict):
    b_obj = b_obj_var(inp_params,variable)
    return pd.DataFrame.from_dict(b_obj.p_summary| b_obj.p_totals,orient='index',columns=[variable['value']]).T

In [ ]:
def df_var_list(inp_params:dict,varlist:dict):
    param = varlist['param']
    value_list = varlist['values']
    return [df_var(inp_params,{'param':param,'value':value}) for value in value_list]

In [ ]:
def mod_inp_params(inp_params,subconfig):
    new_config = inp_params.copy()
    for key,value in subconfig.items():
        new_config[key]=value
    return new_config

In [ ]:
config1 = {'fs':715e3,
           'lout':{'family':'hbed053t',
                        'value(uH)':0.68,
                        'config':'series'},
           'hsfet_partnum':'AONS66408',
           'lsfet_partnum':'AONS66408',
           'q4_partnum':'AONR36368',
           'm_hs':2,
           'm_ls':1,}

In [ ]:
config2 = {'fs':715e3,
           'lout':{'family':'hbed053t',
                        'value(uH)':0.68,
                        'config':'series'},
           'hsfet_partnum':'SIRA74DP',
           'lsfet_partnum':'SIRA74DP',
           'q4_partnum':'SIRA74DP',
           'm_hs':1,
           'm_ls':1,}

In [ ]:
config3 = {'fs':715e3,
           'lout':{'family':'hbed063t',
                        'value(uH)':2.2,
                        'config':'parallel'},
           'hsfet_partnum':'SIRA74DP',
           'lsfet_partnum':'SIRA74DP',
           'q4_partnum':'SIRA74DP',
           'm_hs':1,
           'm_ls':1,}

In [ ]:
config4 = {'fs':600e3,
           'lout':{'family':'hbed063t',
                        'value(uH)':2.2,
                        'config':'parallel'},
           'hsfet_partnum':'SIS488DN',
           'lsfet_partnum':'SISA72ADN',
           'q4_partnum':'AON7318',
           'm_hs':2,
           'm_ls':2,}

In [ ]:
config5 = {'fs':600e3,
           'lout':{'family':'hbed063t',
                        'value(uH)':2.2,
                        'config':'parallel'},
           'hsfet_partnum':'SISS4410DN',
           'lsfet_partnum':'SISA72ADN',
           'q4_partnum':'AON7318',
           'm_hs':2,
           'm_ls':2,}

In [ ]:

new_params1 = mod_inp_params(input_params,config1)
new_params2 = mod_inp_params(input_params,config2)
new_params3 = mod_inp_params(input_params,config3)
new_params4 = mod_inp_params(input_params,config4)
new_params5 = mod_inp_params(input_params,config5)

In [ ]:
iout = range(0,15,1)
var_list = {'param':'iout','values':iout}

In [ ]:
df1=pd.concat(df_var_list(new_params1,var_list))
df2=pd.concat(df_var_list(new_params2,var_list))
df3=pd.concat(df_var_list(new_params3,var_list))
df4=pd.concat(df_var_list(new_params4,var_list))
df5=pd.concat(df_var_list(new_params5,var_list))

In [ ]:
df1['config']='config1'
df2['config']='config2'
df3['config']='config3'
df4['config']='config4'
df5['config']='config5'

In [ ]:
df_all=pd.concat([df1,df2,df3,df4,df5])

In [ ]:
df_all

In [ ]:
#df['total'].plot.line()
df_all.drop([0]).groupby('config')['efficiency'].plot(legend=True)

In [ ]:
#df['total'].plot.line()
df_all.groupby('config')['total'].plot(legend=True)

In [ ]:
df_all.to_csv(r'sim_results\140W_28Vin_10Vout_multipleconfigs_100523.csv')

In [ ]:
duh.p_totals

In [ ]:
b_loss.p_summary

In [ ]:
b_loss.p_totals,b_loss.efficiency

In [ ]:
b_loss.p_summary | b_loss.p_totals

In [ ]:
df = pd.DataFrame.from_dict(b_loss.p_summary| b_loss.p_totals,orient='index',columns=[b_loss.idc])
df.T